# Assignment 6 - Speeding up with Cython

## Using Cython to speed the code

In [1]:
%load_ext Cython

In [2]:
%%cython

import cython

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)

cdef double cy_trapz(double (*f)(double), double a, double b, int n):
    #initializing the variables
    cdef double res = 0   #stores the final result
    cdef double h = (b - a) / n    #width of the interval in the x-axis
    cdef double x2, y1, y2
    y1 = f(a)
    x2 = a + h
    for _ in range(1, n + 1):
        y2 = f(x2)
        
        #increment res by the area of the trapezium
        res += 0.5 * h * (y1 + y2)

        y1 = y2
        x2 += h

    return res

#making the function to integrate inline
@cython.inline
cdef inline double f1(double x):
    return x**2

#using a wrapper function to measure the time taken
def trap_wrapper():
    #10 million trapezoids
    return cy_trapz(f1, 0, 1, 10**7)

In [3]:
%%timeit 

trap_wrapper()

13.1 ms ± 381 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Using Numpy to implement the trapezoidal rule of Integration

In [4]:
import numpy as np

In [5]:
def f2(x):
    return x**2

#the interval of points in the x axis
x = np.linspace(0, 1, 10_000_000)

#function values at each x
y = f2(x)

In [6]:
%%timeit

np.trapz(y, x)

98.8 ms ± 1.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Using Python without any libraries for optimizations 

In [7]:
def py_trapz(f, a, b, n):
    #similar functionality to cy_trapz
    h = (b - a) / n
    res = 0

    y1 = f(a)
    x2 = a + h
    for _ in range(1, n + 1):
        y2 = f(x2)
        
        res += 0.5 * h * (y1 + y2)

        y1 = y2
        x2 += h

    return res


def f3(x):
    return x**2

In [8]:
%%timeit

py_trapz(f3, 0, 1, 10**7)

1.98 s ± 29.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
